In [2]:
import os
import pandas as pd

from testing.tpch import setup

QUERIES = setup.QUERIES
COLUMN_MAP = setup.COLUMN_MAP

RESULTS_PATH = os.curdir + "/results/single-queries/tpch/2025-05-10-15H/"

results_df = pd.read_csv(RESULTS_PATH + '/expanded_results.csv')

if "Unnamed: 0" in results_df.columns:
    results_df = results_df.drop(columns=["Unnamed: 0"])

results_df

,Query,Materialization,Execution Time,Previous Time,Improvement,Previous Materializations
0,q3,c_custkey,2.534976,2.679544,0.144568,[]
1,q7,c_custkey,6.707714,7.334910,0.627196,[]
2,q8,c_custkey,6.064164,6.669960,0.605796,[]
3,q10,c_custkey,2.638145,3.381033,0.742888,[]
4,q13,c_custkey,1.216612,2.806873,1.590261,[]
...,...,...,...,...,...,...
16164,q2,s_acctbal,6.126290,6.192996,0.066707,"['s_nationkey', 's_comment']"
16165,q2,s_comment,6.126290,6.228790,0.102501,"['s_nationkey', 's_acctbal']"
16166,q2,s_phone,7.599417,7.609972,0.010554,"['s_acctbal', 's_comment']"
16167,q2,s_acctbal,7.599417,7.603652,0.004235,"['s_phone', 's_comment']"


In [3]:
# Add table size to results
scale_factor = 0.5
tpch_table_sizes = {
    "customer": int(150000 * scale_factor),
    "lineitem": int(6000000 * scale_factor),
    "orders": int(1500000 * scale_factor),
    "part": int(200000 * scale_factor),
    "partsupp": int(800000 * scale_factor),
    "supplier": int(10000 * scale_factor),
    "nation": 25,  # Not scaled
    "region": 5    # Not scaled
}


def get_table_size(column_name):
    if column_name.startswith("c_"):
        return tpch_table_sizes["customer"]
    elif column_name.startswith("l_"):
        return tpch_table_sizes["lineitem"]
    elif column_name.startswith("o_"):
        return tpch_table_sizes["orders"]
    elif column_name.startswith("p_"):
        return tpch_table_sizes["part"]
    elif column_name.startswith("ps_"):
        return tpch_table_sizes["partsupp"]
    elif column_name.startswith("s_"):
        return tpch_table_sizes["supplier"]
    elif column_name.startswith("n_"):
        return tpch_table_sizes["nation"]
    elif column_name.startswith("r_"):
        return tpch_table_sizes["region"]
    else:
        return 0
    
def get_table_name(column_name):
    if column_name.startswith("c_"):
        return "customer"
    elif column_name.startswith("l_"):
        return "lineitem"
    elif column_name.startswith("o_"):
        return "orders"
    elif column_name.startswith("p_"):
        return "part"
    elif column_name.startswith("ps_"):
        return "partsupp"
    elif column_name.startswith("s_"):
        return "supplier"
    elif column_name.startswith("n_"):
        return "nation"
    elif column_name.startswith("r_"):
        return "region"
    else:
        return "unknown"
    
results_df["Table size"] = results_df.apply(lambda row: get_table_size(row["Materialization"]), axis=1)

In [4]:
# Add rank and percentile within query and globally
# Sort by query and time gain, then add rank within each query group
results_df['Query Rank'] = results_df.groupby('Query')['Improvement'].rank(ascending=False).astype(int)

# Calculate percentage rank within each query group (0-100%)
results_df['Query Percentile'] = results_df.groupby('Query')['Improvement'].rank(pct=True).round(2)

# Sort by query and time gain, then add rank within each query group
results_df['Global Rank'] = results_df['Improvement'].rank(ascending=False).astype(int)

# Calculate percentage rank within each query group (0-100%)
results_df['Global Percentile'] = results_df['Improvement'].rank(pct=True).round(2)


In [5]:
# Sort the dataframe by improvement
results_df.sort_values("Improvement", ascending=False, inplace=True)

In [6]:
# Add query usage frequency
def get_field_frequency_for_query(query_name, materialized_column):
    q = QUERIES[query_name]
    cols = q.columns_used()
    return cols.count(materialized_column)

def get_field_join_frequency_for_query(query_name, materialized_column):
    q = QUERIES[query_name]
    cols = q.columns_used_in_join()
    if materialized_column in cols:
        return len(cols[materialized_column])
    else:
        return 0


def get_field_where_frequency_for_query(query_name, materialized_column):
    q = QUERIES[query_name]
    cols = q.columns_used_with_position()["where"]
    return cols.count(materialized_column)

def get_field_select_frequency_for_query(query_name, materialized_column):
    q = QUERIES[query_name]
    cols = q.columns_used_with_position()["select"]
    return cols.count(materialized_column)

def get_field_group_by_frequency_for_query(query_name, materialized_column):
    q = QUERIES[query_name]
    cols = q.columns_used_with_position()["group_by"]
    return cols.count(materialized_column)

def get_field_order_by_frequency_for_query(query_name, materialized_column):
    q = QUERIES[query_name]
    cols = q.columns_used_with_position()["order_by"]
    return cols.count(materialized_column)

def get_self_join_frequency_for_query(query_name, materialized_column):
    q = QUERIES[query_name]
    cols = q.columns_used_with_position()
    if "self_join" in cols and materialized_column in cols["self_join"].keys():
        return cols["self_join"][materialized_column]
    return 0


results_df["Total Frequency"] = results_df.apply(
    lambda row: get_field_frequency_for_query(row["Query"], row["Materialization"]), axis=1
)

results_df["Join Frequency"] = results_df.apply(
    lambda row: get_field_join_frequency_for_query(row["Query"], row["Materialization"]), axis=1
)

results_df["Where Frequency"] = results_df.apply(
    lambda row: get_field_where_frequency_for_query(row["Query"], row["Materialization"]), axis=1
)

results_df["Select Frequency"] = results_df.apply(
    lambda row: get_field_select_frequency_for_query(row["Query"], row["Materialization"]), axis=1
)
results_df["Group By Frequency"] = results_df.apply(
    lambda row: get_field_group_by_frequency_for_query(row["Query"], row["Materialization"]), axis=1
)
results_df["Order By Frequency"] = results_df.apply(
    lambda row: get_field_order_by_frequency_for_query(row["Query"], row["Materialization"]), axis=1
)

results_df["Self Join Frequency"] = results_df.apply(
    lambda row: get_self_join_frequency_for_query(row["Query"], row["Materialization"]), axis=1
)


In [7]:
results_df[(results_df["Query"] == 'q11') & (results_df["Materialization"] == 'ps_suppkey')]

,Query,Materialization,Execution Time,Previous Time,Improvement,Previous Materializations,Table size,Query Rank,Query Percentile,Global Rank,Global Percentile,Total Frequency,Join Frequency,Where Frequency,Select Frequency,Group By Frequency,Order By Frequency,Self Join Frequency
15545,q11,ps_suppkey,1.686540,3.320677,1.634138,"['ps_availqty', 's_suppkey']",400000,1,1.00,406,0.97,2,2,0,0,0,0,0
15575,q11,ps_suppkey,1.736682,3.346493,1.609811,"['ps_supplycost', 's_suppkey']",400000,2,1.00,410,0.97,2,2,0,0,0,0,0
11787,q11,ps_suppkey,3.027366,4.634965,1.607599,"['n_nationkey', 'ps_availqty']",400000,3,0.99,411,0.97,2,2,0,0,0,0,0
15288,q11,ps_suppkey,2.993184,4.595092,1.601908,"['ps_partkey', 'ps_supplycost']",400000,4,0.99,415,0.97,2,2,0,0,0,0,0
15557,q11,ps_suppkey,2.968470,4.570017,1.601547,"['ps_availqty', 's_nationkey']",400000,5,0.98,416,0.97,2,2,0,0,0,0,0
15276,q11,ps_suppkey,3.053127,4.614507,1.561380,"['ps_partkey', 'ps_availqty']",400000,6,0.98,439,0.97,2,2,0,0,0,0,0
1744,q11,ps_suppkey,1.557714,3.115757,1.558044,['n_name'],400000,7,0.97,442,0.97,2,2,0,0,0,0,0
11191,q11,ps_suppkey,1.564803,3.119265,1.554462,"['n_nationkey', 'n_name']",400000,8,0.97,445,0.97,2,2,0,0,0,0,0
11799,q11,ps_suppkey,3.062749,4.615436,1.552687,"['n_nationkey', 'ps_supplycost']",400000,9,0.97,451,0.97,2,2,0,0,0,0,0
15590,q11,ps_suppkey,2.976709,4.529327,1.552618,"['ps_supplycost', 's_nationkey']",400000,10,0.96,452,0.97,2,2,0,0,0,0,0


## Negative Values
Some of the materializations give negative results

In [8]:
treshold = -0.05
negative_improvement_df = results_df[results_df["Improvement"] < treshold]
print(f'There are {len(negative_improvement_df)} with improvement less than {treshold}')
print(f'The queries with negative improvmement are {negative_improvement_df["Query"].unique()}')
print(f'The fields whose improvement was negative are {negative_improvement_df["Materialization"].unique()}')

There are 457 with improvement less than -0.05
The queries with negative improvmement are ['q7' 'q9' 'q8' 'q2' 'q10' 'q20' 'q3' 'q21' 'q11' 'q18' 'q17']
The fields whose improvement was negative are ['n_nationkey' 'n_name' 'l_discount' 'l_suppkey' 'c_comment' 'p_mfgr'
 's_comment' 's_address' 'o_orderkey' 'r_regionkey' 'l_extendedprice'
 'p_partkey' 'ps_suppkey' 'o_custkey' 'o_orderdate' 's_name' 's_nationkey'
 's_suppkey' 's_acctbal' 's_phone' 'l_orderkey' 'c_custkey' 'l_quantity'
 'ps_availqty' 'ps_supplycost' 'o_shippriority' 'c_name' 'ps_partkey']


# Only Single Joins

In [9]:
single_join_df = results_df[(results_df["Join Frequency"] == 1) & (results_df["Where Frequency"] == 0)]
sj_negative_improvement_df = single_join_df[single_join_df["Improvement"] < treshold]
print(single_join_df[single_join_df["Global Percentile"]>0.8]["Query"].unique())

['q21' 'q9' 'q18' 'q7' 'q13' 'q8' 'q2' 'q10' 'q3' 'q20' 'q12' 'q16' 'q22']


In [10]:
sj_q18 = single_join_df[(single_join_df["Query"] == 'q18') & (single_join_df["Materialization"] == 'l_orderkey')]

# sj_q18 = single_join_df[single_join_df["Query"] == 'q3']
sj_q18.head()

,Query,Materialization,Execution Time,Previous Time,Improvement,Previous Materializations,Table size,Query Rank,Query Percentile,Global Rank,Global Percentile,Total Frequency,Join Frequency,Where Frequency,Select Frequency,Group By Frequency,Order By Frequency,Self Join Frequency
4713,q18,l_orderkey,2.443687,4.154700,1.711014,"['c_name', 'o_orderkey']",3000000,15,0.94,394,0.98,2,1,0,1,0,0,0
2913,q18,l_orderkey,1.710876,3.410222,1.699346,"['c_custkey', 'o_orderkey']",3000000,16,0.94,396,0.98,2,1,0,1,0,0,0
6416,q18,l_orderkey,1.641986,3.311369,1.669384,"['o_custkey', 'o_orderkey']",3000000,17,0.93,400,0.98,2,1,0,1,0,0,0
5669,q18,l_orderkey,0.844736,2.513315,1.668578,"['l_quantity', 'o_orderkey']",3000000,18,0.93,401,0.98,2,1,0,1,0,0,0
6347,q18,l_orderkey,1.689167,3.346678,1.657511,"['o_orderdate', 'o_orderkey']",3000000,19,0.92,402,0.98,2,1,0,1,0,0,0


In [11]:
sj_q18.tail()

,Query,Materialization,Execution Time,Previous Time,Improvement,Previous Materializations,Table size,Query Rank,Query Percentile,Global Rank,Global Percentile,Total Frequency,Join Frequency,Where Frequency,Select Frequency,Group By Frequency,Order By Frequency,Self Join Frequency
6392,q18,l_orderkey,2.221494,3.507505,1.286012,"['o_totalprice', 'o_custkey']",3000000,52,0.78,1153,0.93,2,1,0,1,0,0,0
450,q18,l_orderkey,4.229903,5.508631,1.278728,['c_name'],3000000,53,0.78,1166,0.93,2,1,0,1,0,0,0
2877,q18,l_orderkey,3.521224,4.797943,1.276719,"['c_custkey', 'o_orderdate']",3000000,54,0.77,1175,0.93,2,1,0,1,0,0,0
4707,q18,l_orderkey,3.610531,4.875039,1.264508,"['c_name', 'o_custkey']",3000000,55,0.77,1198,0.93,2,1,0,1,0,0,0
2898,q18,l_orderkey,2.891103,4.113966,1.222863,"['c_custkey', 'o_custkey']",3000000,60,0.75,1282,0.92,2,1,0,1,0,0,0


In [12]:
print(f'There are {len(sj_negative_improvement_df)} with improvement less than {treshold}')
print(f'The queries with negative improvmement are {sj_negative_improvement_df["Query"].unique()}')
print(f'The fields whose improvement was negative are {sj_negative_improvement_df["Materialization"].unique()}')

There are 173 with improvement less than -0.05
The queries with negative improvmement are ['q8' 'q9' 'q20' 'q2' 'q21' 'q18' 'q7' 'q3']
The fields whose improvement was negative are ['o_orderkey' 'r_regionkey' 'p_partkey' 'ps_suppkey' 'o_custkey'
 's_nationkey' 'n_nationkey' 'c_custkey' 's_suppkey' 'l_orderkey']


# Single Where

In [13]:
single_where_df = results_df[(results_df["Join Frequency"] == 0) & (results_df["Where Frequency"] == 1)]
print(single_where_df[single_where_df["Global Percentile"]>0.8]["Query"].unique())

['q7' 'q12' 'q2' 'q18' 'q21' 'q8' 'q3' 'q19' 'q20' 'q16' 'q10' 'q9' 'q4']


In [55]:
single_where_query_df = single_where_df[single_where_df["Query"] == 'q16']
single_where_query_df = single_where_query_df[single_where_query_df["Materialization"] == 'ps_suppkey']
# single_where_query_df[single_where_query_df["Materialization"] == 'o_orderstatus'].head()
# single_where_query_df[single_where_query_df["Previous Materializations"] == "['l_commitdate']"]
# single_where_query_df[single_where_query_df["Previous Materializations"] == "[]"]
# single_where_query_df[single_where_query_df["Materialization"] == 'p_container']
single_where_query_df
# TODO 16 - ps_suppkey

,Query,Materialization,Execution Time,Previous Time,Improvement,Previous Materializations,Table size,Query Rank,Query Percentile,Global Rank,Global Percentile,Total Frequency,Join Frequency,Where Frequency,Select Frequency,Group By Frequency,Order By Frequency,Self Join Frequency
15259,q16,ps_suppkey,0.785678,0.828740,0.043062,"['p_brand', 'ps_partkey']",400000,127,0.46,9376,0.42,2,0,1,1,0,0,0
15306,q16,ps_suppkey,1.585430,1.616666,0.031236,"['ps_partkey', 's_suppkey']",400000,131,0.44,9970,0.38,2,0,1,1,0,0,0
15345,q16,ps_suppkey,0.901260,0.930501,0.029241,"['ps_partkey', 's_comment']",400000,133,0.43,10087,0.38,2,0,1,1,0,0,0
2198,q16,ps_suppkey,1.591179,1.620196,0.029016,['ps_partkey'],400000,134,0.43,10107,0.37,2,0,1,1,0,0,0
14898,q16,ps_suppkey,1.524820,1.551267,0.026448,"['p_size', 's_suppkey']",400000,137,0.41,10284,0.36,2,0,1,1,0,0,0
13993,q16,ps_suppkey,1.529330,1.554262,0.024932,"['p_type', 'ps_partkey']",400000,139,0.41,10371,0.36,2,0,1,1,0,0,0
15270,q16,ps_suppkey,0.896484,0.918385,0.021902,"['p_brand', 's_comment']",400000,141,0.40,10569,0.35,2,0,1,1,0,0,0
14064,q16,ps_suppkey,1.604965,1.624652,0.019686,"['p_type', 's_comment']",400000,142,0.39,10781,0.33,2,0,1,1,0,0,0
14848,q16,ps_suppkey,0.771932,0.790408,0.018476,"['p_size', 'ps_partkey']",400000,144,0.38,10892,0.33,2,0,1,1,0,0,0
2130,q16,ps_suppkey,1.518253,1.536415,0.018162,['p_size'],400000,145,0.38,10926,0.32,2,0,1,1,0,0,0


In [32]:
# single_where_query_df[single_where_df["Materialization"] == 'o_orderstatus'].tail(25)
single_where_query_df.tail(25)

,Query,Materialization,Execution Time,Previous Time,Improvement,Previous Materializations,Table size,Query Rank,Query Percentile,Global Rank,Global Percentile,Total Frequency,Join Frequency,Where Frequency,Select Frequency,Group By Frequency,Order By Frequency,Self Join Frequency
10604,q3,l_shipdate,1.247725,1.903712,0.655986,"['o_orderdate', 'o_shippriority']",3000000,135,0.71,6166,0.62,1,0,1,0,0,0,0
5976,q3,l_shipdate,1.090563,1.746215,0.655652,"['l_orderkey', 'o_orderkey']",3000000,136,0.71,6173,0.62,1,0,1,0,0,0,0
1365,q3,l_shipdate,1.273636,1.925454,0.651819,['o_custkey'],3000000,139,0.70,6252,0.61,1,0,1,0,0,0,0
9276,q3,l_shipdate,1.254562,1.905996,0.651434,"['l_extendedprice', 'o_orderdate']",3000000,140,0.70,6262,0.61,1,0,1,0,0,0,0
9894,q3,l_shipdate,1.303841,1.953608,0.649767,"['l_discount', 'o_custkey']",3000000,141,0.70,6304,0.61,1,0,1,0,0,0,0
10613,q3,l_shipdate,1.284867,1.932470,0.647603,"['o_shippriority', 'o_custkey']",3000000,142,0.69,6344,0.61,1,0,1,0,0,0,0
430,q3,l_shipdate,1.362823,2.009757,0.646934,['c_mktsegment'],3000000,143,0.69,6361,0.61,1,0,1,0,0,0,0
1090,q3,l_shipdate,2.058407,2.704213,0.645806,['l_extendedprice'],3000000,144,0.69,6383,0.61,1,0,1,0,0,0,0
3282,q3,l_shipdate,1.996251,2.641778,0.645527,"['c_custkey', 'o_shippriority']",3000000,145,0.69,6387,0.61,1,0,1,0,0,0,0
10610,q3,l_shipdate,1.260966,1.906023,0.645057,"['o_orderdate', 'o_orderkey']",3000000,146,0.68,6396,0.60,1,0,1,0,0,0,0


# Things to investigate 
q10, n_nationkey gives best performance when n_name materialized

- q21, n_nationkey negative
- q21, hvorfor er s_nationkey alltid dritbra

In [16]:
# Look 

In [17]:
treshold = 1.5
positive_df = results_df[results_df["Improvement"] > treshold]
print(f'There are {len(positive_df)} with improvement less than {treshold}')
print(f'The queries with negative improvmement are {positive_df["Query"].unique()}')
print(f'The fields whose improvement was negative are {positive_df["Materialization"].unique()}')
positive_df[positive_df['Query'] == 'q20'].head(10)

There are 609 with improvement less than 1.5
The queries with negative improvmement are ['q21' 'q7' 'q18' 'q2' 'q9' 'q11' 'q13' 'q12' 'q17' 'q20']
The fields whose improvement was negative are ['l_orderkey' 'n_name' 's_nationkey' 'o_orderkey' 's_suppkey' 'l_shipdate'
 'o_custkey' 'l_suppkey' 'ps_suppkey' 'ps_partkey' 'c_custkey' 'r_name'
 'n_regionkey' 'l_receiptdate' 'l_partkey']


,Query,Materialization,Execution Time,Previous Time,Improvement,Previous Materializations,Table size,Query Rank,Query Percentile,Global Rank,Global Percentile,Total Frequency,Join Frequency,Where Frequency,Select Frequency,Group By Frequency,Order By Frequency,Self Join Frequency
15308,q20,ps_partkey,2.364814,3.888371,1.523556,"['ps_suppkey', 's_suppkey']",400000,1,1.0,507,0.97,2,1,1,0,0,0,0


# Queries
Print queries and materialization queries

In [52]:
query_names = ['q16']
# materializations = [['l_orderkey'], ['l_orderkey', 'o_custkey'], ['o_orderkey'], ['o_orderkey', 'o_custkey'],['o_totalprice'], ['o_totalprice', 'o_custkey'] ]
# materializations = [['o_custkey', 's_nationkey'], ['o_custkey', 's_nationkey', 'o_orderkey'], ['l_discount', 's_nationkey'], ['l_discount', 's_nationkey', 'o_orderkey']]
# materializations = [[],['s_nationkey']]
materializations = [[], ['p_brand']]

In [53]:
for materialization in materializations:
    for query_name in query_names:
        query_obj = QUERIES[query_name]
        m = 0
        update_stmt = "UPDATE test_table SET "
        print(f"################# {query_name.upper()}, {materialization} #################")
        for field in materialization:
            field_obj = COLUMN_MAP[field]

            print(f"ALTER TABLE test_table DROP COLUMN IF EXISTS {field};")
            print(f"ALTER TABLE test_table ADD {field} {field_obj['type']};")
            update_stmt += f"{field} = {field_obj['access']}, "
        update_stmt = update_stmt[:-2] + ";"
        print(update_stmt)
        # Create the field-materialization setup for this test
        fields = []
        for field, access_query in COLUMN_MAP.items():
            fields.append(
                (field, access_query, field in materialization))
        
        print(QUERIES[query_name].get_query(fields=fields))


################# Q16, [] #################
UPDATE test_table SE;

SELECT
    CAST(p.raw_json->>'p_brand' AS VARCHAR) AS p_brand,
    CAST(p.raw_json->>'p_type' AS VARCHAR) AS p_type,
    CAST(p.raw_json->>'p_size' AS INT) AS p_size,
    COUNT(DISTINCT CAST(ps.raw_json->>'ps_suppkey' AS INT)) AS supplier_cnt
FROM
    test_table ps,
    test_table p
WHERE
    CAST(p.raw_json->>'p_partkey' AS INT) = CAST(ps.raw_json->>'ps_partkey' AS INT)
    AND CAST(p.raw_json->>'p_brand' AS VARCHAR) <> 'Brand#45'
    AND CAST(p.raw_json->>'p_type' AS VARCHAR) NOT LIKE 'MEDIUM POLISHED%'
    AND CAST(p.raw_json->>'p_size' AS INT) IN (49, 14, 23, 45, 19, 3, 36, 9)
    AND CAST(ps.raw_json->>'ps_suppkey' AS INT) NOT IN (
        SELECT
            CAST(s.raw_json->>'s_suppkey' AS INT)
        FROM
            test_table s
        WHERE
            CAST(s.raw_json->>'s_comment' AS VARCHAR) LIKE '%Customer%Complaints%'
    )
GROUP BY
    CAST(p.raw_json->>'p_brand' AS VARCHAR),
    CAST(p.raw_json->>'p_typ